In [ ]:
import pandas as pd
from datetime import datetime

In [ ]:
# Load post metadata (assuming scraped data)
df = pd.read_csv("telegram_posts.csv")  # must contain: ['channel', 'message_id', 'timestamp', 'views', 'clean_text']

# Convert timestamp to datetime
df["timestamp"] = pd.to_datetime(df["timestamp"])
df["week"] = df["timestamp"].dt.isocalendar().week

# Posting frequency
post_freq = df.groupby("channel")["week"].nunique().reset_index(name="Posts_per_Week")

# Avg views
avg_views = df.groupby("channel")["views"].mean().reset_index(name="Avg_Views")

# Highest viewed post
top_posts = df.loc[df.groupby("channel")["views"].idxmax()][["channel", "views", "clean_text"]].rename(columns={"views": "Top_Views", "clean_text": "Top_Post"})

# Extract average price using your NER model
import re

price_pattern = re.compile(r"\b(\d{2,7})\s*ብር\b")

def extract_prices(text):
    return [int(m[0]) for m in price_pattern.findall(text)]

df["prices"] = df["clean_text"].apply(extract_prices)
price_stats = df.explode("prices").groupby("channel")["prices"].mean().reset_index(name="Avg_Price")

# Combine all metrics
scorecard = post_freq.merge(avg_views, on="channel")\
                    .merge(price_stats, on="channel")\
                    .merge(top_posts, on="channel")

# Compute Lending Score
scorecard["Lending_Score"] = (scorecard["Avg_Views"] * 0.5) + (scorecard["Posts_per_Week"] * 0.5)

# Save scorecard
scorecard.to_csv("vendor_scorecard.csv", index=False)
print("✅ Vendor Scorecard saved as vendor_scorecard.csv")